In [1]:
import sys
import pyfftw
import time
import ROOT
from ROOT import gROOT
from ROOT import gStyle
%jsroot on

ROOT.gInterpreter.AddIncludePath('../../include/')
ROOT.gInterpreter.Declare('#include "RoEvent.h"')
ROOT.gSystem.Load('../../libs/libRoEvent.so')

gROOT.LoadMacro("../style/CMS/tdrstyle.C")
gROOT.ProcessLine("setTDRStyle();")
gROOT.LoadMacro("../style/CMS/CMS_lumi.C")

import pickle
from array import array
from pathlib import Path
import numpy as np
from scipy import signal
sys.path.append("../package/")
import lycro_0v1 as lycro

from scipy.signal import find_peaks, peak_widths 

gStyle.SetOptFit(0)

Welcome to JupyROOT 6.24/00


In [2]:
run = 1170; thr = 70; window = 50
pick = "../../tmp/ju_r" + str(run) + "_thr-" + str(thr) + "_wind-" + str(window) + ".pik"
root = "../../tmp/ro_r" + str(run) + ".root"
print("PICK file: {}".format(pick))
print("ROOT file: {}".format(root))

PICK file: ../../tmp/ju_r1170_thr-70_wind-50.pik
ROOT file: ../../tmp/ro_r1170.root


In [3]:
f = ROOT.TFile(root)
myTree = f.Get("data") 

#loking signals
ro_signals = []
numEv = myTree.GetEntries()
for entry in myTree:
    events = entry.events
    numCh = events.getNumPayloads()
    ev = events.getHeader().getEvtNum()
    print("Ev {}:{} is processing... ".format(ev, numEv), end='\r')
    for n in range(numCh):
        
        card = events.getPayloads().AddrAt(n).getMap().getCard()
        ch = events.getPayloads().AddrAt(n).getCh()

        cro = np.asarray(events.getPayloads().AddrAt(n).getCRO())
        numSa = events.getPayloads().AddrAt(n).getNsaCRO()
        ped = lycro.findPedestal(data = cro, window = window, thr = 30, offset = 5000)
        pos_sigs = lycro.findSignals(cro, round(np.average(ped)), thr = 70, window = window, pol = 1, run=0, ev = ev, card = card, ch = ch)
        neg_sigs = lycro.findSignals(cro, round(np.average(ped)), thr = 70, window = window, pol = -1, run=0, ev = ev, card = card, ch = ch)
        ro_signals.extend(pos_sigs)
        ro_signals.extend(neg_sigs)
with open(pick, "wb") as f:
    pickle.dump(len(ro_signals), f)
    for value in ro_signals:
        pickle.dump(value, f)